In [1]:
import requests
from PyPDF2 import PdfReader
import faiss
import pickle
from sentence_transformers import SentenceTransformer

C:\Users\Shree\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:

def fetch_pdf_content(pdf_url):
    try:
        response = requests.get(pdf_url)  # Fetch PDF from URL
        with open("temp.pdf", "wb") as f:
            f.write(response.content)  # Save the PDF locally
        
        reader = PdfReader("temp.pdf")  # Load the PDF
        content = ""
        for page in reader.pages:
            content += page.extract_text()  # Extract text from all pages
        
        return content
    except Exception as e:
        print(f"Error fetching content from {pdf_url}: {e}")
        return None

print("PDF fetching function ready!")

PDF fetching function ready!


In [3]:
def create_faiss_index(articles):
    model = SentenceTransformer('all-MiniLM-L6-v2')  
    embeddings = model.encode(articles)
    index = faiss.IndexFlatL2(len(embeddings[0]))  
    index.add(embeddings) 
    return index, embeddings

print("FAISS index creation function ready!")

FAISS index creation function ready!


In [4]:
def save_faiss_index(index, embeddings, file_name="faiss_store.pkl"):
    with open(file_name, "wb") as f:
        pickle.dump({"index": index, "embeddings": embeddings}, f)
    print(f"FAISS index and embeddings saved to {file_name}!")

def load_faiss_index(file_name="faiss_store.pkl"):
    with open(file_name, "rb") as f:
        data = pickle.load(f)
    return data["index"], data["embeddings"]

print("Save and load functions ready!")


Save and load functions ready!


In [5]:
def get_answer(question, articles, index, model):
    query_embedding = model.encode([question])  
    distances, indices = index.search(query_embedding, k=1)
    answer_index = indices[0][0]  
    return articles[answer_index]

print("Question-answering system ready!")

Question-answering system ready!


In [6]:
urls = [
    "https://mohua.gov.in/upload/uploadfiles/files/PMSVANidhi%20Guideline_English.pdf"
]

articles = [fetch_pdf_content(url) for url in urls if url]

if articles and articles[0]:
    print("Sample Article Content:\n", articles[0][:500]) 

    print("Creating FAISS index...")
    faiss_index, embeddings = create_faiss_index(articles)
    save_faiss_index(faiss_index, embeddings) 
    print("FAISS index created and saved!")
else:
    print("No valid articles fetched. Check your URLs.")


Sample Article Content:
 Scheme guidelines for
AtmaNirbhar Bharat
A SpeciAl Micro-credit FAcility For Street VendorS
MAking Street VendorS SelF-reliAntPM Street Vendor’s AtmaNirbhar Nidhi
(PM SVANidhi)1.   Background
Street vendors represent a very important 
constituent of the urban informal economy and play a significant role in ensuring availability of the goods and services at affordable rates at the door-step of the city dwellers. They 
are known as vendors, hawkers, thelewala, 
rehriwala, theliphadwala etc. in dif
Creating FAISS index...
FAISS index and embeddings saved to faiss_store.pkl!
FAISS index created and saved!


In [8]:
faiss_index, embeddings = load_faiss_index()

model = SentenceTransformer('all-MiniLM-L6-v2')

question = "What are the Scheme Application Process of PMSVANidhi?"

if articles and articles[0]:
    answer = get_answer(question, articles, faiss_index, model)
    print("\nQuestion:", question)
    print("Answer:", answer)
else:
    print("No articles available for question answering.")



Question: What are the Scheme Application Process of PMSVANidhi?
Answer: Scheme guidelines for
AtmaNirbhar Bharat
A SpeciAl Micro-credit FAcility For Street VendorS
MAking Street VendorS SelF-reliAntPM Street Vendor’s AtmaNirbhar Nidhi
(PM SVANidhi)1.   Background
Street vendors represent a very important 
constituent of the urban informal economy and play a significant role in ensuring availability of the goods and services at affordable rates at the door-step of the city dwellers. They 
are known as vendors, hawkers, thelewala, 
rehriwala, theliphadwala etc. in different areas/ contexts. The goods supplied by them include vegetables, fruits, ready-to-eat street food, tea, pakodas, breads, eggs, textile, apparel, footwear, artisan products, books/ stationary etc. The services include barber shops, cobblers, pan shops, laundry services etc. The COVID-19 pandemic and consequent lockdowns have adversely impacted the livelihoods of street vendors. They usually 
work with a small capital 